<a href="https://colab.research.google.com/github/MicheleUIT/gen-ai-photography/blob/main/image_improver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed May 15 15:15:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0              31W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
pip install diffusers transformers accelerate scipy safetensors accelerate

In [3]:
pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/


In [4]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

model_id = "stabilityai/stable-diffusion-2-1"

# Use the DPMSolverMultistepScheduler (DPM-Solver++) scheduler here instead
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

prompt = "a photo of an astronaut riding a horse on mars"
image = pipe(prompt).images[0]

image.save("astronaut_rides_horse.png")
image

AttributeError: module 'jax.random' has no attribute 'KeyArray'

In [ ]:
import requests
from PIL import Image

import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration

model_id = "llava-hf/llava-1.5-7b-hf"

prompt = "USER: <image>\nDescribe what could be improved in this picture\nASSISTANT:"


model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    load_in_4bit=True
)

processor = AutoProcessor.from_pretrained(model_id)


raw_image = image
inputs = processor(prompt, raw_image, return_tensors='pt').to(0, torch.float16)

output = model.generate(**inputs, max_new_tokens=200, do_sample=False)
print(processor.decode(output[0][2:], skip_special_tokens=True))
# https://huggingface.co/llava-hf/llava-1.5-7b-hf

In [ ]:
!pip install -qq diffusers transformers ftfy gradio accelerate
!pip install -qq "ipywidgets>=7,<8"

In [ ]:
import inspect
import warnings
from typing import List, Optional, Union
import torch
from torch import autocast
from tqdm.auto import tqdm
import gradio as gr
from diffusers import StableDiffusionImg2ImgPipeline

device = "cuda"
model_path = "CompVis/stable-diffusion-v1-4"

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_path,
    revision="fp16",
    torch_dtype=torch.float16,
)
pipe = pipe.to(device)

In [ ]:
def predict(image, prompt):
  style ='gta san andreas cover art, borderlands style, celshading, trending on artstation, by rhads, andreas rocha, rossdraws, makoto shinkai, laurie greasley, lois van baarle, ilya kuvshinov and greg rutkowski' #@param ['None', 'gta san andreas cover art, borderlands style, celshading, trending on artstation, by rhads, andreas rocha, rossdraws, makoto shinkai, laurie greasley, lois van baarle, ilya kuvshinov and greg rutkowski', 'Symmetry!! greek mythology, ancient greece, intricate, elegant, highly detailed, cinematic lighting, digital art, digital painting, artstation, concept art, sharp focus, illustration, art by artgerm and greg rutkowski and alphonse mucha, 8 k'] {type:'string'}
  if style is not None:
    prompt = prompt + ", " + style
  init_img =  image.resize((512, 512))
#   init_img = preprocess(init_img)
  with autocast("cuda"):
    image = pipe(prompt=prompt, init_image=init_img, strength=0.65, guidance_scale=7.5).images[0]

  return(image)

gr.Interface(
    predict,
    title = 'Image to image',
    inputs=[
        gr.Image(label="Image prompt",type = 'pil'),
        gr.Textbox(label = 'Text prompt')
    ],
    outputs = [
        gr.Image()
        ]
).launch(debug=True)